In [21]:
import pandas as pd 
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.preview.generative_models as generative_models

def generate(input_text, safety_settings, generation_config):
    vertexai.init(project="mtrx-wg2-modeling-dev-9yj", location="us-east1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
    )
    responses = model.generate_content(
      [input_text],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
    )
    
    resText = ""
    for response in responses:
        resText+=response.text
        
    return resText


#############################################
## GEMINI STUFF #############################
#############################################
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
]

drugs_to_indications = pd.read_excel("../indicationList.xlsx")

indicationsData = list(drugs_to_indications['indications'])
activeIngredientsData = list(drugs_to_indications['active ingredient'])
print(len(indicationsData), ' indications sections found')




50378  indications sections found


In [24]:
#############################################
## MAIN SECTION #############################
#############################################

diseasesTreated = []
therapyActiveIngredients = []
originalText = []
for index, item in enumerate(indications_data):
    if index < 100:
        try:
            input_text = "Produce a list of diseases treated in the following therapeutic indications list:\n" + item +  "Please format the list as [\'item1\', \'item2\', ... ,\'itemN\']. Do not inlude any other text in the response. If no diseases are treated, return an empty list as \'[]\'. If the therapy is preventative, add the tag (preventative) to the item. If the drug is only used for diagnostic purposes, return \'diagnostic/contrast/radiolabel\'."
            response = generate(input_text, safety_settings, generation_config)
            diseasesTreated.append(response)
            therapyActiveIngredients.append(activeIngredientsData[index])
            originalText.append(item)
        except:
            diseasesTreated.append("LLM ingest returned error")
            therapyActiveIngredients.append(activeIngredientsData[index])
            originalText.append(item)


data = pd.DataFrame({'active ingredient(s)':therapyActiveIngredients,'original text':originalText, 'diseases treated': diseasesTreated})
data.to_excel("drugs_to_diseases_test.xlsx")
            

I0000 00:00:1724693019.048265  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693020.161277  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693020.845617  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693021.581760  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693022.279684  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693022.893350  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693023.955242  947423 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1724693024.744078  947423 check_gcp_

In [25]:

reingested_data = pd.read_excel("drugs_to_diseases_test.xlsx")
print(reingested_data)

    Unnamed: 0                  active ingredient(s)  \
0            0                      {'POLYTHIAZIDE'}   
1            1                        {'METOLAZONE'}   
2            2                        {'TOLAZAMIDE'}   
3            3                       {'DIATRIZOATE'}   
4            4                        {'MEPERIDINE'}   
..         ...                                   ...   
95          95                         {'TORSEMIDE'}   
96          96                         {'MUPIROCIN'}   
97          97  {'FLUOROMETHOLONE', 'SULFACETAMIDE'}   
98          98                         {'QUINIDINE'}   
99          99     {'PROPOXYPHENE', 'ACETAMINOPHEN'}   

                                        original text  \
0   Renese is indicated as adjunctive therapy in e...   
1   MYKROX Tablets are indicated for the treatment...   
2   TOLINASE Tablets are indicated as an adjunct t...   
3   HYPAQUE-76 is indicated for excretory urograph...   
4   Meperidine is indicated for the relief

In [29]:
print(reingested_data['diseases treated'][2])

['noninsulin dependent diabetes mellitus (Type II)', 'hyperglycemia', 'long-term cardiovascular complications of diabetes (preventative)', 'neural complications of diabetes (preventative)'] 



In [35]:
x = reingested_data['active ingredient(s)'][17]

In [36]:
print(x)

{'DEXTROSE', 'POTASSIUM CHLORIDE', 'SODIUM GLUCONATE', 'SODIUM ACETATE', 'MAGNESIUM CHLORIDE', 'SODIUM CHLORIDE'}


In [37]:
print(len(x))

113
